In [1]:
import wrds
import pytz
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime as dt
import time
import random

In [2]:
from numpy.linalg import det, inv, eigvals
from scipy.optimize import minimize, LinearConstraint, Bounds

In [3]:
with open('password_wrds.txt') as f:
    password = f.readlines()

In [4]:
# Connect to WRDS
conn = wrds.Connection()

Enter your WRDS username [prith]: prithwish
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [5]:
# Query to get historical S&P 500 constituents
query = """
SELECT DISTINCT dsp500.permno, dsp500.start, dsp500.ending, msenames.ticker, msenames.comnam
FROM crsp.dsp500list AS dsp500
INNER JOIN crsp.msenames AS msenames
ON dsp500.permno = msenames.permno
WHERE dsp500.start < '1996-01-01' and dsp500.ending > '2020-12-31';
"""

# Execute the query and fetch the data
sp500_constituents = conn.raw_sql(query)

In [6]:
# Replace 'crsp' and 'dsp500list' with the schema and table name you are interested in
table_description = conn.describe_table('crsp', 'dsf')

# Print the description
print(table_description)

Approximately 105258384 rows in crsp.dsf.
       name  nullable              type comment
0     cusip      True        VARCHAR(8)    None
1    permno      True           INTEGER    None
2    permco      True           INTEGER    None
3    issuno      True           INTEGER    None
4     hexcd      True          SMALLINT    None
5    hsiccd      True           INTEGER    None
6      date      True              DATE    None
7     bidlo      True    NUMERIC(11, 5)    None
8     askhi      True    NUMERIC(11, 5)    None
9       prc      True    NUMERIC(11, 5)    None
10      vol      True    NUMERIC(10, 0)    None
11      ret      True    NUMERIC(10, 6)    None
12      bid      True    NUMERIC(11, 5)    None
13      ask      True    NUMERIC(11, 5)    None
14   shrout      True  DOUBLE PRECISION    None
15   cfacpr      True  DOUBLE PRECISION    None
16  cfacshr      True  DOUBLE PRECISION    None
17  openprc      True    NUMERIC(11, 5)    None
18   numtrd      True           INTEGER    Non

In [7]:
# Query to get historical S&P 500 constituents
start_time = time.time()
query = """
SELECT date, permno, ret, prc, cfacpr
FROM crsp.dsf
WHERE date BETWEEN '1996-01-01' AND '2020-12-31';
"""

# Execute the query and fetch the data
stock_timess = conn.raw_sql(query)
print("--- %s seconds ---" % (time.time() - start_time))
stock_timess

--- 457.11151027679443 seconds ---


,date,permno,ret,prc,cfacpr
0,1996-01-02,10001,-0.026667,-9.12500,1.5
1,1996-01-03,10001,0.041096,9.50000,1.5
2,1996-01-04,10001,-0.039474,-9.12500,1.5
3,1996-01-05,10001,-0.041096,8.75000,1.5
4,1996-01-08,10001,0.000000,8.75000,1.5
...,...,...,...,...,...
8243,2020-12-24,93436,0.024444,661.77002,3.0
8244,2020-12-28,93436,0.002901,663.69000,3.0
8245,2020-12-29,93436,0.003465,665.98999,3.0
8246,2020-12-30,93436,0.043229,694.78003,3.0


In [9]:
#stock_timess.to_pickle('stock_ts_pickle.p')

stock_timess = pd.read_pickle('stock_ts_pickle.p')

In [10]:
# Disconnect from WRDS
conn.close()

In [11]:
special_cases = sp500_constituents[~sp500_constituents['ticker'].isna()][['permno', 'ticker']].drop_duplicates()

# Currently, using a randomly generated subset Of the stocks present in the time frame from 1996 to 2020
random.seed(10)
sdr = random.sample(range(1, special_cases.shape[0]), 120)
myset = special_cases.iloc[sdr]
myset_dict = dict(zip(myset.permno, myset.ticker))

In [12]:
# Define the stock permanent numbers for S&P 500 stocks (replace with actual permnos)
permnos = list(myset_dict.keys())  # Example permnos for AAPL, MSFT, WMT respectively
# Initialize an empty DataFrame to store results
daily_returns = pd.DataFrame()
# Loop through each stock's permno and fetch data
for permno in permnos:
    stock_data= stock_timess[stock_timess['permno']==permno].copy()
    stock_data.rename(columns={'ret': myset_dict[permno]}, inplace=True)
    #stock_data['date'] = pd.to_datetime(stock_data['date'])
    stock_data.set_index('date', inplace=True)
    stock_data.index = pd.to_datetime(stock_data.index).strftime('%Y-%m-%d')
    # Concatenate daily returns for each stock to the main DataFrame
    daily_returns = pd.concat([daily_returns, stock_data[myset_dict[permno]]], axis=1, sort=True)

Yahoo finance data fetch

In [66]:
# Define the stock tickers and the time period
tickers = ["AAPL", "MSFT", "AMZN", "GOOGL", "META", "TSLA", "BRK-B", "JNJ",
"WMT", "V", "PG", "NVDA", "JPM", "UNH", "HD", "MA", "DIS", "PFE", "VZ", 
"INTC", "CMCSA", "T", "KO", "MRK", "PEP", "ABT", "CVX", "ORCL", "CSCO", 
"NFLX", "XOM", "IBM", "NKE", "CRM", "ADBE", "AVGO", "GS", "QCOM", "BA", 
"PYPL", "TXN", "AXP", "AMD", "C", "MS", "CAT", "HON", "SBUX", "COST", "MCD"]  # Replace with S&P 500 tickers

tz = pytz.timezone("America/New_York")
start = tz.localize(dt(1996,1,1))
end = tz.localize(dt(2020,12,31))
start_date = "1996-01-01"
end_date = "2020-12-31"

# Download the stock price data
data = yf.download(tickers, start=start, end=end)['Adj Close']


[*********************100%%**********************]  50 of 50 completed


In [68]:
data = data.drop(columns=['PYPL', 'MA']) # very recenlty released IPOs

In [69]:
daily_returns = data.pct_change(fill_method='ffill')

C:\Users\prith\AppData\Local\Temp\ipykernel_30472\4103529606.py:1: FutureWarning: The 'fill_method' keyword being not None and the 'limit' keyword in DataFrame.pct_change are deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_returns = data.pct_change(fill_method='ffill')


In [43]:
#training and testing dates
flag = 2008

if flag == 2008:
    training_period_start = "1996-01-01"
    training_period_end = "2007-12-31"
    test_period_end = "2008-12-31"
else:
    training_period_start = "2009-01-01"
    training_period_end = "2019-12-31"
    test_period_end = "2020-12-31"

In [44]:
# Calculate the mean and covariance of daily returns for the first 12 years
training_data = daily_returns.loc[training_period_start:training_period_end]

mean_returns = training_data.mean()
cov_matrix = training_data.cov()

In [45]:
# Risk aversion parameter (gamma)
gamma = 10.0

# Vector of ones
n = len(mean_returns)
ones = np.ones(n)

mu = np.array(mean_returns)
Sigma = np.array(cov_matrix)

# Note here, there is sum constraint, but no bound constraints
# Calculate (gamma * Sigma)^-1
gamma_sigma_inv = inv(gamma * Sigma)

# Calculate lambda
lambda_val = (ones.T @ gamma_sigma_inv @ mu - 1) / (ones.T @ gamma_sigma_inv @ ones)

# Calculate optimal portfolio weights
a = gamma_sigma_inv @ (mu - lambda_val * ones)

In [46]:
#check realized variance
testing_data = daily_returns.loc[training_period_end:test_period_end]

mean_returns_t = testing_data.mean()
cov_matrix_t = testing_data.cov()

Sigma_realized = np.array(cov_matrix_t)
Realized_variance = a.T @ Sigma_realized @ a 

In [47]:
no_working_days = testing_data.shape[0]
std_error_rv = np.sqrt(2/(no_working_days))* Realized_variance

print('Realized_variance ', "{:.2e}".format(Realized_variance))
print('2x Std error ', "{:.2e}".format(2* std_error_rv))

Realized_variance  9.21e-04
2x Std error  1.63e-04


In [48]:
#forecast of the future variance = initial time frame variance
forcasted_variance = a.T @ Sigma @ a

no_training_days = training_data.shape[0]
std_error_fv = np.sqrt((2 * a.T @ Sigma @ Sigma @ a) / no_training_days)

print('forcasted_variance ', "{:.2e}".format(forcasted_variance))
print('2x Std error ', "{:.2e}".format(2*std_error_fv))

forcasted_variance  1.44e-04
2x Std error  3.19e-05


In [49]:
I = np.eye(n)

def objective_a_oftheta(a, theta):
    term1 = 1 / np.sqrt(det(I - theta * gamma * np.outer(a,a) @ Sigma))
    return (1/theta)*np.log(term1) + a.T @ mu

def positive_definite_constraint(a, theta):
    eigenvalues = eigvals(inv(Sigma) - theta * gamma * np.outer(a,a) )
    return np.min(eigenvalues) - 1e-10  # Ensure all eigenvalues are slightly greater than zero

def worst_case_covariance_matrix(Sigma, theta, gamma, a):
    inv_Sigma_worst_case = inv(Sigma) - theta * gamma * np.outer(a, a)
    return inv(inv_Sigma_worst_case)

In [65]:
#specific case for analysis
theta_ = 700

In [66]:
# sampled possible initial points
n_points = 2
points_on_hyperplane = np.zeros((n_points, n))

for j in range(n_points):
    random_numbers = np.random.rand(n)
    # Normalize the numbers so their sum is 10
    normalized_numbers = random_numbers / random_numbers.sum() 

    points_on_hyperplane[j] = normalized_numbers

In [67]:
# Initial guesses for 'a'
initial_guesses_if = []
for a0 in points_on_hyperplane:
    initial_guesses_if.extend([np.random.permutation(a0) for _ in range(100)])
initial_guesses = [aa for aa in initial_guesses_if if positive_definite_constraint(aa, theta_)>0]

In [68]:
len(initial_guesses_if), len(initial_guesses)

(200, 200)

In [69]:
start_time = time.time()

#extra constraints
ones_for_sum = np.ones((1, n))
linear_constraint = LinearConstraint(ones_for_sum, [1], [1])
bounds = Bounds(0, 1)
# Initialize variables to track the best result
best_result = None

# Iterate over each initial guess
for ax in initial_guesses:

    # Define constraints in the format required by 'minimize'
    constraints = [{'type': 'ineq', 'fun': positive_definite_constraint, 'args': (theta_,)},
                  linear_constraint]
    
    # Perform the optimization
    result = minimize(objective_a_oftheta, ax, method='SLSQP', constraints=constraints, bounds=None, args = (theta_))
    if ~result.success:
        print("Optimization failed:", result.message)
    # Update the best result if necessary
    if best_result is None or result.fun < best_result.fun:
        best_result = result
        
# Check if the optimization was successful
if best_result.success:
    a_star = best_result.x
    print("Optimized a:", a_star)
    print(" Minimum objective reached at", np.real(best_result.fun), "Not portfolio objective!")
else:
    print("Optimization failed:", best_result.message)

print("--- %s Optimization time :seconds ---" % (time.time() - start_time))

Optimized a: [-0.02480483  0.02340253  0.01173643 -0.02585991  0.05361257  0.04009559
  0.04928721 -0.01526163  0.05560254  0.03725308  0.03444988 -0.01726011
 -0.00888254  0.04588782 -0.05565864  0.03174566 -0.0152721   0.04441597
  0.0040984   0.02069922  0.00375933  0.00527893  0.0423104   0.01252786
  0.04320106 -0.01260059  0.06218643 -0.08984915 -0.01563911  0.04827752
  0.02677956  0.01087718  0.05182509 -0.04551886 -0.04058754  0.08833358
  0.04570204  0.08690696 -0.01274729 -0.04780483 -0.05239957  0.01798185
 -0.00646307  0.01306903  0.06347882  0.1559514   0.0094484   0.04586273
 -0.04177039  0.01859627  0.09109784  0.0065718   0.00204922  0.01257142
 -0.01513821  0.04557855 -0.14862356 -0.0375305   0.01550181 -0.02006664
  0.00248279 -0.00965163 -0.02394949  0.05055536  0.0215898   0.02957425
 -0.05834488 -0.04168184 -0.0009507  -0.03180758  0.04223558  0.07964388
 -0.01249482  0.0444535  -0.04516207  0.00812321  0.00891845 -0.00952687
 -0.00690359  0.04796783 -0.00262453  

In [70]:
Sigma_tilda = worst_case_covariance_matrix(Sigma, theta_, gamma, a_star)
worst_variance = a.T @ Sigma_tilda @ a
std_error_wv = np.sqrt((2 * a.T @ Sigma_tilda @ Sigma_tilda @ a) / no_training_days)

print('worst_variance ', "{:.2e}".format(worst_variance))
print('std_error_wv ', "{:.2e}".format(std_error_wv))

Model_Error = np.abs(forcasted_variance - worst_variance)
print('Model_Error ', "{:.2e}".format(Model_Error))

worst_variance  1.44e-04
std_error_wv  1.62e-05
Model_Error  6.57e-08


In [71]:
print('forcasted variance with both errors in confidence interval-')
t_err = 2*(std_error_wv+Model_Error)
print('(',"{:.2e}".format(worst_variance-t_err), "{:.2e}".format(worst_variance+t_err), ')')

forcasted variance with both errors in confidence interval-
( 1.12e-04 1.77e-04 )


In [57]:
print("{:.2e}".format(t_err))

3.31e-05
